In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch
from transformers import GPT2LMHeadModel

In [3]:
import numpy as np
import pandas as pd
import torch
import re

In [4]:
data = pd.read_csv("/content/drive/MyDrive/gdsc/KOSPI_연결개별.csv")
data

,회사명,거래소코드,회계년도,상장일,상장폐지일,소속코드,유동비율,당좌비율,현금비율,순운전자본비율,...,당좌자산회전률,부가가치,총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,폐지사유
0,(주)DB하이텍,990,2011/12,1975/12/12,NaN,1,35.20,25.50,5.04,-42.95,...,2.54,217837.57,18.17,29.75,7.890000e+01,39.45,62.53,37.47,-44.96,NaN
1,(주)DB하이텍,990,2012/12,1975/12/12,NaN,1,55.66,41.39,7.19,-21.27,...,2.73,245655.23,21.22,37.03,1.122200e+02,41.58,55.93,44.07,-12.76,NaN
2,(주)DB하이텍,990,2013/12,1975/12/12,NaN,1,46.76,31.58,7.39,-23.99,...,2.60,191554.48,18.10,31.86,1.124500e+02,38.80,67.15,32.85,-43.35,NaN
3,(주)DB하이텍,990,2014/12,1975/12/12,NaN,1,65.77,47.39,10.72,-13.08,...,3.45,187341.51,19.02,33.99,1.369700e+02,33.00,68.59,31.41,-41.11,NaN
4,(주)DB하이텍,990,2015/12,1975/12/12,NaN,1,138.40,104.20,44.31,9.52,...,3.08,386127.30,39.21,78.88,2.302400e+02,57.93,35.93,64.07,32.82,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8819,흥아해운(주),3280,2018/12,1976/06/29,NaN,1,34.67,28.66,4.89,-25.40,...,8.14,34054.90,4.24,6.91,1.135163e+09,4.52,201.90,-101.90,-254.12,NaN
8820,흥아해운(주),3280,2019/12,1976/06/29,NaN,1,46.36,42.55,6.75,-25.18,...,1.16,13366.25,3.10,5.59,4.455416e+08,13.08,241.39,-141.39,-384.23,NaN
8821,흥아해운(주),3280,2020/12,1976/06/29,NaN,1,38.67,33.73,3.21,-32.68,...,1.29,-1108.19,-0.39,-0.62,0.000000e+00,-1.25,0.00,0.00,0.00,NaN
8822,흥아해운(주),3280,2021/12,1976/06/29,NaN,1,207.21,181.62,143.19,11.56,...,1.63,56593.70,22.88,39.81,0.000000e+00,69.25,41.38,58.62,29.04,NaN


In [5]:
data_c = data.copy()

In [6]:
data_c = data_c.drop(['상장일', '상장폐지일', '폐지사유'], axis = 1)

In [7]:
data_c

,회사명,거래소코드,회계년도,소속코드,유동비율,당좌비율,현금비율,순운전자본비율,부채비율,자기자본배율,...,유동자산회전률,당좌자산회전률,부가가치,총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율
0,(주)DB하이텍,990,2011/12,1,35.20,25.50,5.04,-42.95,289.22,143.54,...,1.87,2.54,217837.57,18.17,29.75,7.890000e+01,39.45,62.53,37.47,-44.96
1,(주)DB하이텍,990,2012/12,1,55.66,41.39,7.19,-21.27,298.00,130.71,...,2.01,2.73,245655.23,21.22,37.03,1.122200e+02,41.58,55.93,44.07,-12.76
2,(주)DB하이텍,990,2013/12,1,46.76,31.58,7.39,-23.99,416.90,92.00,...,1.86,2.60,191554.48,18.10,31.86,1.124500e+02,38.80,67.15,32.85,-43.35
3,(주)DB하이텍,990,2014/12,1,65.77,47.39,10.72,-13.08,715.95,54.23,...,2.41,3.45,187341.51,19.02,33.99,1.369700e+02,33.00,68.59,31.41,-41.11
4,(주)DB하이텍,990,2015/12,1,138.40,104.20,44.31,9.52,287.57,114.15,...,2.28,3.08,386127.30,39.21,78.88,2.302400e+02,57.93,35.93,64.07,32.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8819,흥아해운(주),3280,2018/12,1,34.67,28.66,4.89,-25.40,998.73,83.29,...,6.73,8.14,34054.90,4.24,6.91,1.135163e+09,4.52,201.90,-101.90,-254.12
8820,흥아해운(주),3280,2019/12,1,46.36,42.55,6.75,-25.18,1293.83,55.16,...,1.01,1.16,13366.25,3.10,5.59,4.455416e+08,13.08,241.39,-141.39,-384.23
8821,흥아해운(주),3280,2020/12,1,38.67,33.73,3.21,-32.68,0.00,-45.03,...,1.16,1.29,-1108.19,-0.39,-0.62,0.000000e+00,-1.25,0.00,0.00,0.00
8822,흥아해운(주),3280,2021/12,1,207.21,181.62,143.19,11.56,140.24,85.63,...,1.43,1.63,56593.70,22.88,39.81,0.000000e+00,69.25,41.38,58.62,29.04


In [8]:
data_c = data_c.dropna()

In [9]:
data_c.isna().sum()

회사명             0
거래소코드           0
회계년도            0
소속코드            0
유동비율            0
당좌비율            0
현금비율            0
순운전자본비율         0
부채비율            0
자기자본배율          0
이자보상배율(이자비용)    0
비유동비율           0
유동자산구성비율        0
자기자본구성비율        0
차입금의존도          0
매출액증가율          0
총자본증가율          0
자기자본증가율         0
순이익증가율          0
유형자산증가율         0
비유동자산증가율        0
유동자산증가율         0
재고자산증가율         0
매출액총이익률         0
매출액순이익률         0
총자본사업이익률        0
총자본순이익률         0
자기자본순이익률        0
1주당순이익          0
1주당매출액          0
금융비용 대 총비용비율    0
총자본회전률          0
자기자본회전률         0
비유동자산회전률        0
재고자산회전률         0
매출채권회전률         0
매입채무회전률         0
매입채무회전기간        0
재고자산회전기간        0
매출채권회전기간        0
1회전기간           0
운전자본회전률         0
순운전자본회전률        0
유형자산회전율         0
유동자산회전률         0
당좌자산회전률         0
부가가치            0
총자본투자효율         0
설비투자효율          0
기계투자효율          0
부가가치율           0
노동소득분배율         0
자본분배율           0
이윤분배율           0
dtype: int64

In [10]:
data_c['회계년도'] = pd.to_datetime(data_c['회계년도'])

<ipython-input-10-6d03e79721b8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_c['회계년도'] = pd.to_datetime(data_c['회계년도'])


In [11]:
data_n = data_c[['거래소코드', '소속코드', '유동비율', '당좌비율', '현금비율', '순운전자본비율', '부채비율', '자기자본배율', '이자보상배율(이자비용)', '비유동비율', '유동자산구성비율', '자기자본구성비율', '차입금의존도', '매출액증가율', '총자본증가율', '자기자본증가율', '순이익증가율', '유형자산증가율', '비유동자산증가율', '유동자산증가율', '재고자산증가율', '매출액총이익률', '매출액순이익률', '총자본사업이익률', '총자본순이익률', '1주당순이익', '1주당매출액', '금융비용 대 총비용비율', '총자본회전률', '자기자본회전률', '비유동자산회전률', '재고자산회전률', '매출채권회전률', '매입채무회전률', '매입채무회전기간', '재고자산회전기간', '매출채권회전기간', '1회전기간', '운전자본회전률', '순운전자본회전률', '유형자산회전율', '유동자산회전률', '당좌자산회전률', '부가가치', '총자본투자효율', '설비투자효율', '기계투자효율', '부가가치율', '노동소득분배율', '자본분배율', '이윤분배율']].astype(float)

In [12]:
data_n.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8610 entries, 0 to 8823
Data columns (total 51 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   거래소코드         8610 non-null   float64
 1   소속코드          8610 non-null   float64
 2   유동비율          8610 non-null   float64
 3   당좌비율          8610 non-null   float64
 4   현금비율          8610 non-null   float64
 5   순운전자본비율       8610 non-null   float64
 6   부채비율          8610 non-null   float64
 7   자기자본배율        8610 non-null   float64
 8   이자보상배율(이자비용)  8610 non-null   float64
 9   비유동비율         8610 non-null   float64
 10  유동자산구성비율      8610 non-null   float64
 11  자기자본구성비율      8610 non-null   float64
 12  차입금의존도        8610 non-null   float64
 13  매출액증가율        8610 non-null   float64
 14  총자본증가율        8610 non-null   float64
 15  자기자본증가율       8610 non-null   float64
 16  순이익증가율        8610 non-null   float64
 17  유형자산증가율       8610 non-null   float64
 18  비유동자산증가율      8610 non-null 

In [13]:
data_n.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8610 entries, 0 to 8823
Data columns (total 51 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   거래소코드         8610 non-null   float64
 1   소속코드          8610 non-null   float64
 2   유동비율          8610 non-null   float64
 3   당좌비율          8610 non-null   float64
 4   현금비율          8610 non-null   float64
 5   순운전자본비율       8610 non-null   float64
 6   부채비율          8610 non-null   float64
 7   자기자본배율        8610 non-null   float64
 8   이자보상배율(이자비용)  8610 non-null   float64
 9   비유동비율         8610 non-null   float64
 10  유동자산구성비율      8610 non-null   float64
 11  자기자본구성비율      8610 non-null   float64
 12  차입금의존도        8610 non-null   float64
 13  매출액증가율        8610 non-null   float64
 14  총자본증가율        8610 non-null   float64
 15  자기자본증가율       8610 non-null   float64
 16  순이익증가율        8610 non-null   float64
 17  유형자산증가율       8610 non-null   float64
 18  비유동자산증가율      8610 non-null 

In [14]:
data_n

,거래소코드,소속코드,유동비율,당좌비율,현금비율,순운전자본비율,부채비율,자기자본배율,이자보상배율(이자비용),비유동비율,...,유동자산회전률,당좌자산회전률,부가가치,총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율
0,990.0,1.0,35.20,25.50,5.04,-42.95,289.22,143.54,-0.75,298.42,...,1.87,2.54,217837.57,18.17,29.75,7.890000e+01,39.45,62.53,37.47,-44.96
1,990.0,1.0,55.66,41.39,7.19,-21.27,298.00,130.71,-0.31,291.73,...,2.01,2.73,245655.23,21.22,37.03,1.122200e+02,41.58,55.93,44.07,-12.76
2,990.0,1.0,46.76,31.58,7.39,-23.99,416.90,92.00,-0.16,408.01,...,1.86,2.60,191554.48,18.10,31.86,1.124500e+02,38.80,67.15,32.85,-43.35
3,990.0,1.0,65.77,47.39,10.72,-13.08,715.95,54.23,0.67,610.83,...,2.41,3.45,187341.51,19.02,33.99,1.369700e+02,33.00,68.59,31.41,-41.11
4,990.0,1.0,138.40,104.20,44.31,9.52,287.57,114.15,2.50,254.62,...,2.28,3.08,386127.30,39.21,78.88,2.302400e+02,57.93,35.93,64.07,32.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8819,3280.0,1.0,34.67,28.66,4.89,-25.40,998.73,83.29,-1.16,950.60,...,6.73,8.14,34054.90,4.24,6.91,1.135163e+09,4.52,201.90,-101.90,-254.12
8820,3280.0,1.0,46.36,42.55,6.75,-25.18,1293.83,55.16,-0.55,1090.42,...,1.01,1.16,13366.25,3.10,5.59,4.455416e+08,13.08,241.39,-141.39,-384.23
8821,3280.0,1.0,38.67,33.73,3.21,-32.68,0.00,-45.03,-0.38,0.00,...,1.16,1.29,-1108.19,-0.39,-0.62,0.000000e+00,-1.25,0.00,0.00,0.00
8822,3280.0,1.0,207.21,181.62,143.19,11.56,140.24,85.63,-1.06,186.55,...,1.43,1.63,56593.70,22.88,39.81,0.000000e+00,69.25,41.38,58.62,29.04


In [15]:
data_n_t = data_n.apply(lambda x: f"{x['거래소코드']} 거래소코드, {x['소속코드']} 소속코드, {x['유동비율']} 유동비율, {x['당좌비율']} 당좌비율, {x['현금비율']} 현금비율, {x['순운전자본비율']} 순운전자본비율, {x['부채비율']} 부채비율, {x['자기자본배율']} 자기자본배율, {x['이자보상배율(이자비용)']} 이자비용, {x['비유동비율']} 비유동비율, {x['유동자산구성비율']} 유동자산구성비율, {x['자기자본구성비율']} 자기자본구성비율, {x['차입금의존도']} 차입금의존도, {x['매출액증가율']} 매출액증가율, {x['총자본증가율']} 총자본증가율, {x['자기자본증가율']} 자기자본증가율, {x['순이익증가율']} 순이익증가율, {x['유형자산증가율']} 유형자산증가율, {x['비유동자산증가율']} 비유동자산증가율, {x['유동자산증가율']} 유동자산증가율, {x['재고자산증가율']} 재고자산증가율, {x['매출액총이익률']} 매출액총이익률, {x['매출액순이익률']} 매출액순이익률, {x['총자본사업이익률']} 총자본사업이익률, {x['총자본순이익률']} 총자본순이익률, {x['1주당순이익']} 1주당순이익, {x['1주당매출액']} 1주당매출액, {x['금융비용 대 총비용비율']} 금융비용 대 총비용비율, {x['총자본회전률']} 총자본회전률, {x['자기자본회전률']} 자기자본회전률, {x['비유동자산회전률']} 비유동자산회전률, {x['재고자산회전률']} 재고자산회전률, {x['매출채권회전률']} 매출채권회전률, {x['매입채무회전률']} 매입채무회전률, {x['매입채무회전기간']} 매입채무회전기간, {x['재고자산회전기간']} 재고자산회전기간, {x['매출채권회전기간']} 매출채권회전기간, {x['1회전기간']} 1회전기간, {x['운전자본회전률']} 운전자본회전률, {x['순운전자본회전률']} 순운전자본회전률, {x['유형자산회전율']} 유형자산회전율, {x['유동자산회전률']} 유동자산회전률, {x['당좌자산회전률']} 당좌자산회전률, {x['부가가치']} 부가가치, {x['총자본투자효율']} 총자본투자효율, {x['설비투자효율']} 설비투자효율, {x['기계투자효율']} 기계투자효율, {x['부가가치율']} 부가가치율, {x['노동소득분배율']} 노동소득분배율, {x['자본분배율']} 자본분배율, {x['이윤분배율']} 이윤분배율", axis = 1)

In [16]:
data_n_t

0       990.0 거래소코드, 1.0 소속코드, 35.2 유동비율, 25.5 당좌비율, 5...
1       990.0 거래소코드, 1.0 소속코드, 55.66 유동비율, 41.39 당좌비율,...
2       990.0 거래소코드, 1.0 소속코드, 46.76 유동비율, 31.58 당좌비율,...
3       990.0 거래소코드, 1.0 소속코드, 65.77 유동비율, 47.39 당좌비율,...
4       990.0 거래소코드, 1.0 소속코드, 138.4 유동비율, 104.2 당좌비율,...
                              ...                        
8819    3280.0 거래소코드, 1.0 소속코드, 34.67 유동비율, 28.66 당좌비율...
8820    3280.0 거래소코드, 1.0 소속코드, 46.36 유동비율, 42.55 당좌비율...
8821    3280.0 거래소코드, 1.0 소속코드, 38.67 유동비율, 33.73 당좌비율...
8822    3280.0 거래소코드, 1.0 소속코드, 207.21 유동비율, 181.62 당좌...
8823    3280.0 거래소코드, 1.0 소속코드, 159.54 유동비율, 145.53 당좌...
Length: 8610, dtype: object

In [17]:
data_d = data_c.copy()
data_d['year'] = data_c['회계년도'].dt.year

In [18]:
data_d

,회사명,거래소코드,회계년도,소속코드,유동비율,당좌비율,현금비율,순운전자본비율,부채비율,자기자본배율,...,당좌자산회전률,부가가치,총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,year
0,(주)DB하이텍,990,2011-12-01,1,35.20,25.50,5.04,-42.95,289.22,143.54,...,2.54,217837.57,18.17,29.75,7.890000e+01,39.45,62.53,37.47,-44.96,2011
1,(주)DB하이텍,990,2012-12-01,1,55.66,41.39,7.19,-21.27,298.00,130.71,...,2.73,245655.23,21.22,37.03,1.122200e+02,41.58,55.93,44.07,-12.76,2012
2,(주)DB하이텍,990,2013-12-01,1,46.76,31.58,7.39,-23.99,416.90,92.00,...,2.60,191554.48,18.10,31.86,1.124500e+02,38.80,67.15,32.85,-43.35,2013
3,(주)DB하이텍,990,2014-12-01,1,65.77,47.39,10.72,-13.08,715.95,54.23,...,3.45,187341.51,19.02,33.99,1.369700e+02,33.00,68.59,31.41,-41.11,2014
4,(주)DB하이텍,990,2015-12-01,1,138.40,104.20,44.31,9.52,287.57,114.15,...,3.08,386127.30,39.21,78.88,2.302400e+02,57.93,35.93,64.07,32.82,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8819,흥아해운(주),3280,2018-12-01,1,34.67,28.66,4.89,-25.40,998.73,83.29,...,8.14,34054.90,4.24,6.91,1.135163e+09,4.52,201.90,-101.90,-254.12,2018
8820,흥아해운(주),3280,2019-12-01,1,46.36,42.55,6.75,-25.18,1293.83,55.16,...,1.16,13366.25,3.10,5.59,4.455416e+08,13.08,241.39,-141.39,-384.23,2019
8821,흥아해운(주),3280,2020-12-01,1,38.67,33.73,3.21,-32.68,0.00,-45.03,...,1.29,-1108.19,-0.39,-0.62,0.000000e+00,-1.25,0.00,0.00,0.00,2020
8822,흥아해운(주),3280,2021-12-01,1,207.21,181.62,143.19,11.56,140.24,85.63,...,1.63,56593.70,22.88,39.81,0.000000e+00,69.25,41.38,58.62,29.04,2021


In [19]:
data_e = data_d[['year', '회사명']]

In [20]:
data_e

,year,회사명
0,2011,(주)DB하이텍
1,2012,(주)DB하이텍
2,2013,(주)DB하이텍
3,2014,(주)DB하이텍
4,2015,(주)DB하이텍
...,...,...
8819,2018,흥아해운(주)
8820,2019,흥아해운(주)
8821,2020,흥아해운(주)
8822,2021,흥아해운(주)


In [21]:
data_q = data_e.apply(lambda x: f"{x['year']}년 {x['회사명']} ", axis = 1).astype(str)

In [22]:
data_q

0       2011년 (주)DB하이텍 
1       2012년 (주)DB하이텍 
2       2013년 (주)DB하이텍 
3       2014년 (주)DB하이텍 
4       2015년 (주)DB하이텍 
             ...       
8819     2018년 흥아해운(주) 
8820     2019년 흥아해운(주) 
8821     2020년 흥아해운(주) 
8822     2021년 흥아해운(주) 
8823     2022년 흥아해운(주) 
Length: 8610, dtype: object

In [23]:
data_chat = pd.concat([data_q, data_n_t], axis = 1)

In [24]:
data_chat.columns = ['question', 'answer']
data_chat

,question,answer
0,2011년 (주)DB하이텍,"990.0 거래소코드, 1.0 소속코드, 35.2 유동비율, 25.5 당좌비율, 5..."
1,2012년 (주)DB하이텍,"990.0 거래소코드, 1.0 소속코드, 55.66 유동비율, 41.39 당좌비율,..."
2,2013년 (주)DB하이텍,"990.0 거래소코드, 1.0 소속코드, 46.76 유동비율, 31.58 당좌비율,..."
3,2014년 (주)DB하이텍,"990.0 거래소코드, 1.0 소속코드, 65.77 유동비율, 47.39 당좌비율,..."
4,2015년 (주)DB하이텍,"990.0 거래소코드, 1.0 소속코드, 138.4 유동비율, 104.2 당좌비율,..."
...,...,...
8819,2018년 흥아해운(주),"3280.0 거래소코드, 1.0 소속코드, 34.67 유동비율, 28.66 당좌비율..."
8820,2019년 흥아해운(주),"3280.0 거래소코드, 1.0 소속코드, 46.36 유동비율, 42.55 당좌비율..."
8821,2020년 흥아해운(주),"3280.0 거래소코드, 1.0 소속코드, 38.67 유동비율, 33.73 당좌비율..."
8822,2021년 흥아해운(주),"3280.0 거래소코드, 1.0 소속코드, 207.21 유동비율, 181.62 당좌..."


In [25]:
data_chat.to_csv('data_chat.csv')

In [26]:
from torch.utils.data import Dataset, DataLoader

In [27]:
%%writefile dataloader.py
import pandas as pd
from torch.utils.data import Dataset, DataLoader
class ChatbotDataset(Dataset):
    def __init__(self, tokenizer, file_path):
        data = pd.read_csv(file_path, encoding = "utf-8")
        concats = [label + "|" + text for label, text in zip(data['question'], data['answer'])]
        self.items = [tokenizer(item, return_tensors = "pt", padding = "max_length", truncation = True, max_length = 70)["input_ids"] for item in concats]
        self.length = len(self.items)


    def __getitem__(self, i):
        return self.items[i]
    def __len__(self):
        return self.length

def GPTDataLoader(tokenizer, file_path, batch_size):
    data = ChatbotDataset(tokenizer, file_path)
    return DataLoader(data, batch_size = batch_size)

Overwriting dataloader.py


In [28]:
%%writefile generater.py
import torch
from tqdm import tqdm

def generate(input_text, tokenizer, model, num):
    sentence_list = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    token_ids = tokenizer(input_text + "|", return_tensors = "pt")["input_ids"].to(device)
    for cnt in tqdm(range(num)):
        gen_ids = model.generate(token_ids, max_length = 700, repetition_penalty = 2.0, pad_token_id = tokenizer.pad_token_id, eos_token_id = tokenizer.eos_token_id, bos_token_id = tokenizer.bos_token_id, use_cache = True, do_sample = True)
        sentence = tokenizer.decode(gen_ids[0])
        sentence = sentence[sentence.index("|") + 1 :]
        if "<pad>" in sentence:
            sentence = sentence[: sentence.index("<pad>")].rstrip()
        sentence = sentence.replace("<unk>", " ").split("\n")[0]
        if cnt % 100 == 0 and cnt != 0:
            print(sentence)
        sentence_list.append(sentence)
    return sentence_list

Overwriting generater.py


In [29]:
!pip install torch
!pip install tqdm
!pip install transformers

In [30]:
%%writefile train.py
from google.colab import drive
drive.mount('/content/drive')

import os
import argparse
import torch
from tqdm import tqdm
import sys
import transformers
import pandas as pd
from transformers import AutoTokenizer, get_linear_schedule_with_warmup, AdamW, GPT2LMHeadModel
from dataloader import GPTDataLoader
from generater import generate

if sys.version_info < (3, 7):
    raise Exception("This script requires Python 3.7 or higher.")


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--model_name", default = "skt/kogpt2-base-v2", type = str)
    parser.add_argument("--data_dir", default = "/content/data_chat.csv", type = str)
    parser.add_argument("--batch_size", default = 1, type = int)
    parser.add_argument("--epochs", default = 1, type = int)
    parser.add_argument("--lr", default = 2e-5, type = float)
    parser.add_argument("--warmup_steps", default = 200, type = int)
    args = parser.parse_args('')

    tokenizer = AutoTokenizer.from_pretrained(args.model_name)
    tokenizer.add_special_tokens({"pad_token" : "<pad>"})

    base_dir = os.getcwd()
    data_DIR = os.path.join(base_dir, args.data_dir)

    train_dataloader = GPTDataLoader(tokenizer, data_DIR, args.batch_size)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = GPT2LMHeadModel.from_pretrained(args.model_name).to(device)
    model.train()

    optimizer = AdamW(model.parameters(), lr = args.lr)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = args.warmup_steps, num_training_steps = -1)
    min_loss = int(1e-9)

    for epoch in range(args.epochs):
        print(f"Training epoch {epoch}")
        for input_text in tqdm(train_dataloader):
            input_tensor = input_text.to(device)
            outputs = model(input_tensor, labels = input_tensor)
            loss = outputs[0]
            optimizer.zero_grad()
            model.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

        print(f"epoch {epoch} loss {outputs[0].item():0.2f}")


        model.save_pretrained("chatf_model.h5")

    print("Training Done!")

Overwriting train.py
